In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

#!pip install alpha_vantage
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.fundamentaldata import FundamentalData 
ts = TimeSeries(key = 'HKDDDB1WO21X9QR6', output_format = 'pandas', indexing_type = 'date')
ti = TechIndicators(key = 'HKDDDB1WO21X9QR6', output_format = 'pandas', indexing_type = 'date')
fd = FundamentalData(key = 'HKDDDB1WO21X9QR6', output_format = 'pandas', indexing_type = 'date')

from functools import reduce

In [2]:
ticker = 'ZM'

In [3]:
key = 'HKDDDB1WO21X9QR6'
base_url = "https://www.alphavantage.co/query/"

In [4]:
#build out a request
res = requests.get(url=base_url, params={
    "function":"EARNINGS",
    "symbol":f'{ticker}',
    'apikey':key})

In [5]:
res.status_code

200

In [6]:
EPS = res.json()
# _, header = EPS.json()
#df = pd.DataFrame.from_dict(EPS, orient= 'index')_

In [7]:
len(EPS['quarterlyEarnings'])

7

In [8]:
def quarterly_earnings_df(ticker):
    key = 'HKDDDB1WO21X9QR6'
    base_url = "https://www.alphavantage.co/query/"
    stock = ticker
    res = requests.get(url=base_url, params={
    "function":"EARNINGS",
    "symbol":f'{stock}',
    'apikey':key})
    EPS = res.json()
    
    #Quarterly
    for i in range(0,len(EPS['quarterlyEarnings'])-1):
        if i==0:
            df = pd.DataFrame.from_dict(EPS['quarterlyEarnings'][i], orient = 'index')
        else:
            df_1 = pd.DataFrame.from_dict(EPS['quarterlyEarnings'][i], orient = 'index')
            df = pd.merge(df, df_1, left_index = True, right_index = True) 
    df = df.transpose()
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
    df['reportedDate'] = pd.to_datetime(df['reportedDate'])
    df[['reported_Quarterly_EPS', 'estimated_Quarterly_EPS', 'surprise', 'surprisePercentage']] = df[['reportedEPS', 'estimatedEPS', 'surprise', 'surprisePercentage']].apply(pd.to_numeric, errors = 'coerce')
    df = df.drop(columns = ['estimatedEPS', 'reportedEPS'])
    df = df.set_index('fiscalDateEnding')
    return df

In [9]:
df = quarterly_earnings_df(ticker)

In [10]:
df.head()

,reportedDate,surprise,surprisePercentage,reported_Quarterly_EPS,estimated_Quarterly_EPS
fiscalDateEnding,,,,,
2020-10-31,2020-11-30,0.2296,30.1946,0.99,0.7604
2020-07-31,2020-08-31,0.4662,102.7325,0.92,0.4538
2020-04-30,2020-06-02,0.1015,103.0457,0.20,0.0985
2020-01-31,2020-03-04,0.0789,110.9705,0.15,0.0711
2019-10-31,2019-12-05,0.0589,189.3891,0.09,0.0311


In [11]:
reported_dict = pd.Series(df.reportedDate.values, index = df.index).to_dict()    

In [12]:
def annual_earnings_df(ticker):
    key = 'HKDDDB1WO21X9QR6'
    base_url = "https://www.alphavantage.co/query/"
    stock = ticker
    res = requests.get(url=base_url, params={
    "function":"EARNINGS",
    "symbol":f'{stock}',
    'apikey':key})
    EPS = res.json()
    #Annual
    for i in range(0,len(EPS['annualEarnings'])-1):
        if i==0:
            df = pd.DataFrame.from_dict(EPS['annualEarnings'][i], orient = 'index')
        else:
            df_1 = pd.DataFrame.from_dict(EPS['annualEarnings'][i], orient = 'index')
            df = pd.merge(df, df_1, left_index = True, right_index = True)
    df = df.transpose()
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
    df[['reported_Annual_EPS']] = df[['reportedEPS']].apply(pd.to_numeric, errors = 'coerce')
    df = df.drop(columns = 'reportedEPS')
    df = df.set_index('fiscalDateEnding')
    return df

In [13]:
df = annual_earnings_df(ticker)
#change index to the reported date for annual earnings 
#df.index = df.index.map(reported_dict)
df.head()

,reported_Annual_EPS
fiscalDateEnding,
2020-10-31,2.11


In [14]:
def all_earnings_df(ticker):
    key = 'HKDDDB1WO21X9QR6'
    base_url = "https://www.alphavantage.co/query/"
    stock = ticker
    res = requests.get(url=base_url, params={
    "function":"EARNINGS",
    "symbol":f'{stock}',
    'apikey':key})
    EPS = res.json()
    #Quarterly
    for i in range(0,len(EPS['quarterlyEarnings'])-1):
        if i==0:
            df = pd.DataFrame.from_dict(EPS['quarterlyEarnings'][i], orient = 'index')
        else:
            df_1 = pd.DataFrame.from_dict(EPS['quarterlyEarnings'][i], orient = 'index')
            df = pd.merge(df, df_1, left_index = True, right_index = True) 
    df = df.transpose()
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
    df['reportedDate'] = pd.to_datetime(df['reportedDate'])
    df[['reported_Quarterly_EPS', 'estimated_Quarterly_EPS', 'surprise', 'surprisePercentage']] = df[['reportedEPS', 'estimatedEPS', 'surprise', 'surprisePercentage']].apply(pd.to_numeric, errors = 'coerce')
    df = df.drop(columns = ['estimatedEPS', 'reportedEPS'])
    df = df.set_index('fiscalDateEnding')
    
    #create dictionary with fiscaldateending (key) and reporteddate (value) to map onto annual 
    reported_dict = pd.Series(df.reportedDate.values, index = df.index).to_dict()    
    
    #change index of quarterly to reported
    df = df.set_index('reportedDate')
    
    #Annual 
    res = requests.get(url=base_url, params={
    "function":"EARNINGS",
    "symbol":f'{stock}',
    'apikey':key})
    EPS = res.json()
    #Annual
    for i in range(0,len(EPS['annualEarnings'])-1):
        if i==0:
            dfa = pd.DataFrame.from_dict(EPS['annualEarnings'][i], orient = 'index')
        else:
            dfa_1 = pd.DataFrame.from_dict(EPS['annualEarnings'][i], orient = 'index')
            dfa = pd.merge(dfa, dfa_1, left_index = True, right_index = True)
    dfa = dfa.transpose()
    dfa['fiscalDateEnding'] = pd.to_datetime(dfa['fiscalDateEnding'])
    dfa[['reported_Annual_EPS']] = dfa[['reportedEPS']].apply(pd.to_numeric, errors = 'coerce')
    dfa = dfa.drop(columns = 'reportedEPS')
    dfa = dfa.set_index('fiscalDateEnding')
    
    #map new index values onto annual so that date is the reported date (more relevant to price action)
    dfa.index = dfa.index.map(reported_dict)

    #merge
    merge = pd.merge(df, dfa, how ='left', left_index = True, right_index = True)
    return merge

In [15]:
df = all_earnings_df(ticker)

In [16]:
df.head()

,surprise,surprisePercentage,reported_Quarterly_EPS,estimated_Quarterly_EPS,reported_Annual_EPS
reportedDate,,,,,
2020-11-30,0.2296,30.1946,0.99,0.7604,2.11
2020-08-31,0.4662,102.7325,0.92,0.4538,NaN
2020-06-02,0.1015,103.0457,0.20,0.0985,NaN
2020-03-04,0.0789,110.9705,0.15,0.0711,NaN
2019-12-05,0.0589,189.3891,0.09,0.0311,NaN


In [17]:
#Create funtion with other functions to get both annual and quarterly earnings into one DF

def get_all_earnings(ticker):
    dfq = quarterly_earnings_df(ticker)
    reported_dict = pd.Series(dfq.reportedDate.values, index = dfq.index).to_dict()    
    dfq = dfq.set_index('reportedDate')
    dfa = annual_earnings_df(ticker)
    dfa.index = dfa.index.map(reported_dict)
    merge = pd.merge(dfq, dfa, how ='left', left_index = True, right_index = True)
    return merge

In [18]:
earnings = get_all_earnings(ticker)

In [19]:
earnings.head(20)

,surprise,surprisePercentage,reported_Quarterly_EPS,estimated_Quarterly_EPS,reported_Annual_EPS
reportedDate,,,,,
2020-11-30,0.2296,30.1946,0.99,0.7604,2.11
2020-08-31,0.4662,102.7325,0.92,0.4538,NaN
2020-06-02,0.1015,103.0457,0.20,0.0985,NaN
2020-03-04,0.0789,110.9705,0.15,0.0711,NaN
2019-12-05,0.0589,189.3891,0.09,0.0311,NaN
2019-09-05,0.0656,455.5556,0.08,0.0144,NaN


In [20]:
earnings = earnings.sort_index()

In [21]:
#Export earnings data to data folder
earnings.to_csv(f'data/{ticker}_earnings.csv')

#### Get Daily Stock Data for Ticker along with Technical Indicators and IWO Index Values 

For Ticker:
- Open 
- High
- Low
- Close
- Volume

Techincal Indicators:
- 8 Day Exponential Moving Average
- 21 Day Exponential Moving Average
- 50 Day Simple Moving Average
- 200 Day Simple Moving Average

IWO Index:
- 5 Day Exponential Moving Average
- 10 Day Exponential Moving Average
- 150 Day Exponential Moving Average

In [22]:
# Get daily stock data
daily, meta_data = ts.get_daily(symbol = ticker, outputsize = 'full')
#UNCOMMENT TO MERGE EARNINGS DATA WITH PRICE ACTION ----- merge = pd.merge(daily, df, how ='outer', left_index = True, right_index = True)

#add IWO stock data for 5day EMA, 10day EMA and 150day EMA
data_5, meta_data = ti.get_ema(symbol = 'IWO', interval='daily', time_period = 5, series_type = 'close')
data_10, meta_data = ti.get_ema(symbol = 'IWO', interval='daily', time_period = 10, series_type = 'close')
data_150, meta_data = ti.get_ema(symbol = 'IWO', interval='daily', time_period = 150, series_type = 'close')
dfs = [data_5, data_10, data_150]
trend = reduce(lambda left, right: pd.merge(left, right, on='date', how = 'inner'), dfs)
trend.columns = ['IWO_5d_EMA', 'IWO_10d_EMA', 'IWO_150d_EMA']

#Inner merge so that we only get IWO data for when the ticker has price data as well (in case ticker recently public)
merge = pd.merge(daily, trend, how ='inner',left_index = True, right_index = True)
merge.head()

,1. open,2. high,3. low,4. close,5. volume,IWO_5d_EMA,IWO_10d_EMA,IWO_150d_EMA
date,,,,,,,,
2021-01-26,391.08,395.8199,372.2000,374.39,5151261.0,313.4677,310.5270,250.6857
2021-01-25,388.99,398.8000,378.2700,391.83,5178085.0,313.7916,310.0174,249.8516
2021-01-22,380.99,385.2899,376.3800,383.40,2791585.0,312.7773,308.7280,248.9662
2021-01-21,383.03,387.2100,372.0300,382.81,5180798.0,311.0610,307.0653,248.0636
2021-01-20,399.11,399.5300,378.3187,382.80,5713892.0,310.5865,305.9665,247.2052


In [23]:
merge.tail()

,1. open,2. high,3. low,4. close,5. volume,IWO_5d_EMA,IWO_10d_EMA,IWO_150d_EMA
date,,,,,,,,
2019-04-25,64.74,66.850,62.600,65.0,3863275.0,198.1412,197.7468,191.3282
2019-04-24,71.40,71.500,63.160,63.2,4973529.0,198.0556,197.6211,191.2345
2019-04-23,66.87,74.169,65.550,69.0,6786513.0,197.3048,197.1909,191.1228
2019-04-22,61.00,68.900,59.940,65.7,9949738.0,196.4454,196.7836,191.0167
2019-04-18,65.00,66.000,60.321,62.0,25764659.0,196.8603,197.0432,190.9550


In [24]:
# Create list of parameter dictionaries
def build_params(ticker):
    param_list = [{'EMA_8' : {
    "function":"EMA",
    "interval":"daily",
    "symbol":f'{ticker}',
    "time_period": "8",
    "series_type": "high",
    'apikey':key}}, 
 
    {'EMA_21':{
    "function":"EMA",
    "interval":"daily",
    "symbol":f'{ticker}',
    "time_period": "21",
    "series_type": "high",
    'apikey':key}}, 
                        
    {'SMA_50':{
    "function":"SMA",
    "interval":"daily",
    "symbol":f'{ticker}',
    "time_period": "50",
    "series_type": "high",
    'apikey':key}},
              
    {'SMA_200':{
    "function":"SMA",
    "interval":"daily",
    "symbol":f'{ticker}',
    "time_period": "200",
    "series_type": "high",
    'apikey':key}}
             
    ]
    return param_list

In [25]:
def get_technicals(stock):
    key = 'HKDDDB1WO21X9QR6'
    base_url = "https://www.alphavantage.co/query/"
    ticker = stock
    dfs = []
    cols = []
    param_list = build_params(ticker)
    for i in param_list:    
        for k, v in i.items():
            res = requests.get(url=base_url, params = v)
            res_dict = res.json()
            _, header = res.json()
            df = pd.DataFrame.from_dict(res_dict[header], orient= 'index')
            dfs.append(df)
            cols.append(k)
    df = pd.concat(dfs, axis = 1)
    df.columns = cols
    df = df.apply(pd.to_numeric, errors = 'coerce')
    return df
            

In [26]:
technicals = get_technicals(ticker)
stock_data = pd.merge(merge, technicals, how ='inner',left_index = True, right_index = True)

In [27]:
stock_data.tail()

,1. open,2. high,3. low,4. close,5. volume,IWO_5d_EMA,IWO_10d_EMA,IWO_150d_EMA,EMA_8,EMA_21,SMA_50,SMA_200
2019-05-06,75.01,80.79,74.5000,78.24,2051824.0,201.5516,200.3865,192.1760,75.4788,NaN,NaN,NaN
2019-05-03,75.00,80.25,75.0000,79.18,2590281.0,200.5140,199.6664,192.0223,73.9613,NaN,NaN,NaN
2019-05-02,72.75,75.89,69.6912,75.50,2525285.0,199.1502,198.8720,191.8717,72.1645,NaN,NaN,NaN
2019-05-01,72.72,76.95,70.8159,72.76,3301900.0,199.1790,198.8229,191.7748,71.1000,NaN,NaN,NaN
2019-04-30,68.40,72.52,66.6700,72.47,4113123.0,199.8345,199.0351,191.6930,69.4286,NaN,NaN,NaN


In [28]:
stock_data.head()

,1. open,2. high,3. low,4. close,5. volume,IWO_5d_EMA,IWO_10d_EMA,IWO_150d_EMA,EMA_8,EMA_21,SMA_50,SMA_200
2021-01-26,391.08,395.8199,372.2000,374.39,5151261.0,313.4677,310.5270,250.6857,390.1800,386.8430,402.6153,335.4447
2021-01-25,388.99,398.8000,378.2700,391.83,5178085.0,313.7916,310.0174,249.8516,388.5686,385.9453,403.0949,334.0931
2021-01-22,380.99,385.2899,376.3800,383.40,2791585.0,312.7773,308.7280,248.9662,385.6453,384.6599,403.4079,332.7285
2021-01-21,383.03,387.2100,372.0300,382.81,5180798.0,311.0610,307.0653,248.0636,385.7469,384.5969,404.6316,331.3942
2021-01-20,399.11,399.5300,378.3187,382.80,5713892.0,310.5865,305.9665,247.2052,385.3289,384.3355,407.0050,330.0841


In [29]:
#Sort index by datetime
stock_data = stock_data.sort_index()

In [30]:
#Export stock data to data folder
stock_data.to_csv(f'data/{ticker}_stock_data.csv')